In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

ee.Initialize(project='ee-arzaaan789')


Successfully saved authorization token.


In [4]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("TGB_bat.csv", delimiter='\t')
df = df[df["occurrenceStatus"] == "PRESENT"]
df = df[df['year']>=2022]
# df['eventDate'] = df['eventDate'].str.replace('/','')
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
df = df.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48185/3320037435.py:4: DtypeWarning: Columns (2,10,16,38,39,40,41,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("TGB_bat.csv", delimiter='\t')


In [5]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# ee.Initialize()

# Your dataframe 'df' must have columns: decimalLongitude, decimalLatitude
# Example: df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')

def create_aoi(lon, lat, box_size_km=1):
    """Create approx 1km x 1km square polygon around lon, lat."""
    half_side_deg = box_size_km / 111.32 / 2  # Rough approx degrees per km

    coords = [
        [lon - half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat - half_side_deg]
    ]
    return ee.Geometry.Polygon(coords)

def compute_all_indices(feature, start_date, end_date):
    """Compute spectral indices and LST for one EE Feature (with AOI geometry)."""
    aoi = feature.geometry()

    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())
    s2 = ee.Image(s2).clip(aoi)

    blue = s2.select('B2')
    green = s2.select('B3')
    red = s2.select('B4')
    nir = s2.select('B8')
    swir = s2.select('B11')

    L = 0.5  # SAVI constant

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L).rename('SAVI')
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    bsi = (red.add(blue).subtract(nir.add(swir))).divide(red.add(blue).add(nir).add(swir)).rename('BSI')
    ui = nir.subtract(swir).divide(nir.add(swir)).rename('UI')

    reducers = ee.Reducer.mean()
    scale_10m = 10

    ndvi_mean = ndvi.reduceRegion(reducers, aoi, scale_10m).get('NDVI')
    ndwi_mean = ndwi.reduceRegion(reducers, aoi, scale_10m).get('NDWI')
    ndbi_mean = ndbi.reduceRegion(reducers, aoi, scale_10m).get('NDBI')
    savi_mean = savi.reduceRegion(reducers, aoi, scale_10m).get('SAVI')
    mndwi_mean = mndwi.reduceRegion(reducers, aoi, scale_10m).get('MNDWI')
    ndsi_mean = ndsi.reduceRegion(reducers, aoi, scale_10m).get('NDSI')
    bsi_mean = bsi.reduceRegion(reducers, aoi, scale_10m).get('BSI')
    ui_mean = ui.reduceRegion(reducers, aoi, scale_10m).get('UI')

    # MODIS LST dataset
    modis = (ee.ImageCollection("MODIS/061/MOD11A1")
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select('LST_Day_1km'))

    lst_mean_img = modis.mean().multiply(0.02).clip(aoi)

    lst_mean = lst_mean_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000
    ).get('LST_Day_1km')

    return feature.set({
        'NDVI': ndvi_mean,
        'NDWI': ndwi_mean,
        'NDBI': ndbi_mean,
        'SAVI': savi_mean,
        'MNDWI': mndwi_mean,
        'NDSI': ndsi_mean,
        'BSI': bsi_mean,
        'UI': ui_mean,
        'LST': lst_mean
    })

# Split df into batches
batch_size = 50
batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]

results_list = []

index_names = ['NDVI', 'NDWI', 'NDBI', 'SAVI', 'MNDWI', 'NDSI', 'BSI', 'UI', 'LST']

for batch in tqdm(batches, desc="Processing batches"):
    features = []
    batch_indices = []
    feature_metadata = {}

    for idx, row in batch.iterrows():
        
        aoi = create_aoi(row['decimalLongitude'], row['decimalLatitude'])
        feature = ee.Feature(aoi).set('index', idx)
        features.append(feature)
        batch_indices.append(idx)  # Save the original index
        
        event_date = row['eventDate']
        start_date = (event_date - timedelta(days=15)).strftime('%Y-%m-%d')
        end_date = (event_date + timedelta(days=15)).strftime('%Y-%m-%d')
        feature_metadata[idx] = (start_date, end_date)

    # Create a FeatureCollection from the list of features
    fc = ee.FeatureCollection(features)
    
    # Define wrapper for map to inject per-feature dates
    def map_with_dates(f):
        idx = f.get('index')
        # Use dictionary lookup to get dates for this feature
        date_dict = ee.Dictionary(ee.Dictionary(feature_metadata))
        dates = ee.List(date_dict.get(ee.Number(idx)))
        return compute_all_indices(f, dates.get(0), dates.get(1))

    try:
        result_fc = fc.map(map_with_dates)
        results = result_fc.getInfo()

        rows = []
        for f in results['features']:
            props = f['properties']
            rows.append(props)

    except Exception as e:
        print(f"Error processing batch starting at index {batch.index[0]}: {e}")
        # If there's an error, create placeholder rows with None
        rows = [{'index': i, **{name: None for name in index_names}} for i in batch_indices]

    batch_results_df = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
    results_list.append(batch_results_df)

# Concatenate all batches and sort by original index
all_results_df = pd.concat(results_list).sort_values('index').reset_index(drop=True)

# Merge with original df
df_final = pd.concat([df.reset_index(drop=True), all_results_df.drop(columns=['index'])], axis=1)

print(df_final.head())


Processing batches: 100%|██████████| 140/140 [06:45<00:00,  2.89s/it]

                     species  decimalLatitude  decimalLongitude  eventDate  \
0  Pipistrellus pipistrellus        52.619031         -0.834527 2022-06-22   
1  Pipistrellus pipistrellus        52.619031         -0.834527 2022-06-26   
2  Pipistrellus pipistrellus        52.619031         -0.834527 2022-06-28   
3  Pipistrellus pipistrellus        52.619031         -0.834527 2022-07-25   
4  Pipistrellus pipistrellus        52.619031         -0.834527 2022-06-23   

        BSI         LST     MNDWI      NDBI      NDSI      NDVI      NDWI  \
0 -0.748604  296.632402 -0.485945 -0.364947 -0.485945  0.776133 -0.718489   
1 -0.685741  297.862372 -0.505891 -0.238189 -0.505891  0.672734 -0.660273   
2 -0.685741  297.862372 -0.505891 -0.238189 -0.505891  0.672734 -0.660273   
3 -0.685741  303.103360 -0.505891 -0.238189 -0.505891  0.672734 -0.660273   
4 -0.748604  296.823268 -0.485945 -0.364947 -0.485945  0.776133 -0.718489   

       SAVI        UI  
0  1.164067  0.364947  
1  1.008978  0.23818

In [6]:
df_final.to_csv("tgb_bat_full_data.csv", index=False)
df = pd.read_csv("tgb_bat_full_data.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

df = df.dropna().reset_index(drop=True)

In [7]:
ceratopogonidae = pd.read_csv("Ceratopogonidae.csv", delimiter='\t')
ceratopogonidae = ceratopogonidae[ceratopogonidae["occurrenceStatus"] == "PRESENT"]
ceratopogonidae = ceratopogonidae[ceratopogonidae['year']>=2022]
ceratopogonidae['eventDate'] = pd.to_datetime(ceratopogonidae['eventDate'], format='%Y-%m-%d', errors='coerce')
ceratopogonidae = ceratopogonidae[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
ceratopogonidae = ceratopogonidae.dropna().reset_index(drop=True)

chironomidae = pd.read_csv("Chironomidae.csv", delimiter='\t')
chironomidae = chironomidae[chironomidae["occurrenceStatus"] == "PRESENT"]
chironomidae = chironomidae[chironomidae['year']>=2022]
chironomidae['eventDate'] = pd.to_datetime(chironomidae['eventDate'], format='%Y-%m-%d', errors='coerce')
chironomidae = chironomidae[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
chironomidae = chironomidae.dropna().reset_index(drop=True)

cats = pd.read_csv("cats.csv", delimiter='\t')
cats = cats[cats["occurrenceStatus"] == "PRESENT"]
cats = cats[cats['year']>=2022]
cats['eventDate'] = pd.to_datetime(cats['eventDate'], format='%Y-%m-%d', errors='coerce')
cats = cats[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
cats = cats.dropna().reset_index(drop=True)


wind_turbines = pd.read_excel("REPD_202407_WIND.xlsx")
wind_turbines = wind_turbines[wind_turbines['DEVSTATSHT']=='Operational']
wind_turbines = wind_turbines[['LAT','LNG']]
wind_turbines = wind_turbines.rename(columns={'LAT': 'decimalLatitude', 'LNG': 'decimalLongitude'})
wind_turbines = wind_turbines.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48185/108303835.py:1: DtypeWarning: Columns (39,41,45,48) have mixed types. Specify dtype option on import or set low_memory=False.
  ceratopogonidae = pd.read_csv("Ceratopogonidae.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48185/108303835.py:8: DtypeWarning: Columns (10,39,41,45,48) have mixed types. Specify dtype option on import or set low_memory=False.
  chironomidae = pd.read_csv("Chironomidae.csv", delimiter='\t')


In [8]:
import numpy as np
from sklearn.neighbors import BallTree

# Convert lat/lon to radians for BallTree
bat_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
ceratopogonidae_coords = np.deg2rad(ceratopogonidae[['decimalLatitude', 'decimalLongitude']].values)
chironomidae_coords = np.deg2rad(chironomidae[['decimalLatitude', 'decimalLongitude']].values)
cats_coords = np.deg2rad(cats[['decimalLatitude', 'decimalLongitude']].values)
wind_turbines_coords = np.deg2rad(wind_turbines[['decimalLatitude', 'decimalLongitude']].values)

# Build BallTree using haversine metric
tree_ceratopogonidae = BallTree(ceratopogonidae_coords, metric='haversine')
tree_chironomidae = BallTree(chironomidae_coords, metric='haversine')
tree_cats = BallTree(cats_coords, metric='haversine')
tree_wind_turbines = BallTree(wind_turbines_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['ceratopogonidae_presence'] = 0
df['chironomidae_presence'] = 0
df['cats_presence'] = 0

# Iterate through each bat point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = bat_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # CERATOPOGONIDAE
    idxs = tree_ceratopogonidae.query_radius(point, r=radius)[0]
    for j in idxs:
        ceratopogonidae_date = ceratopogonidae.loc[j, 'eventDate']
        if ceratopogonidae_date.month == event_month and ceratopogonidae_date.year == event_year:
            df.at[i, 'ceratopogonidae_presence'] = 1
            break  # Found at least one match, no need to check further

    # CHIRONOMIDAE
    idxs = tree_chironomidae.query_radius(point, r=radius)[0]
    for j in idxs:
        chironomidae_date = chironomidae.loc[j, 'eventDate']
        if chironomidae_date.month == event_month and chironomidae_date.year == event_year:
            df.at[i, 'chironomidae_presence'] = 1
            break  # Found at least one match, no need to check further
            
    # CATS
    idxs = tree_cats.query_radius(point, r=radius)[0]
    for j in idxs:
        cats_date = cats.loc[j, 'eventDate']
        if cats_date.month == event_month and cats_date.year == event_year:
            df.at[i, 'cats_presence'] = 1
            break  # Found at least one match, no need to check further
        
indices = tree_wind_turbines.query_radius(bat_coords, r=radius)
df['wind_turbines_presence'] = [1 if len(neighbors) > 0 else 0 for neighbors in indices]

100%|██████████| 6876/6876 [00:00<00:00, 9729.44it/s] 


In [ ]:
import osmnx as ox
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from tqdm import tqdm

# Construct GeoDataFrame
df['geometry'] = [Point(xy) for xy in zip(df['decimalLongitude'], df['decimalLatitude'])]
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Define spatial tiling: 0.05 x 0.05 degrees
tile_size = 0.1
gdf['tile_x'] = (gdf['decimalLongitude'] // tile_size).astype(int)
gdf['tile_y'] = (gdf['decimalLatitude'] // tile_size).astype(int)

# Group by tile
grouped = gdf.groupby(['tile_x', 'tile_y'])

# Store results
all_distances = []

for (tile_x, tile_y), group in tqdm(grouped, total=len(grouped)):
    west = tile_x * tile_size - 0.02
    south = tile_y * tile_size - 0.02
    east = (tile_x + 1) * tile_size + 0.02
    north = (tile_y + 1) * tile_size + 0.02
    bbox = (west, south, east, north)

    try:
        G = ox.graph_from_bbox(bbox, network_type='drive_service')
        if len(G.nodes) == 0:
            print(f"[EMPTY GRAPH] bbox: {bbox}, skipping...")
            all_distances.extend([np.nan] * len(group))
            continue

        G_proj = ox.project_graph(G)
        nodes_proj, edges_proj = ox.graph_to_gdfs(G_proj)
        points_proj = group.geometry.to_crs(nodes_proj.crs)

        for point_proj in points_proj:
            try:
                u, v, k = ox.distance.nearest_edges(G_proj, [point_proj.x], [point_proj.y])[0]
                edge_geom = edges_proj.loc[(u, v, k)]['geometry']
                distance = point_proj.distance(edge_geom)
                all_distances.append(distance)
            except Exception as e:
                print(f"  [Point ERROR] {e}")
                all_distances.append(np.nan)

    except Exception as e:
        print(f"[TILE ERROR] {e} — bbox: {bbox}")
        all_distances.extend([np.nan] * len(group))


# Store back in original DataFrame order
gdf['distance_to_road'] = all_distances

 36%|███▌      | 283/792 [17:11<44:38,  5.26s/it]  

In [ ]:
gdf['near_road'] = np.where(gdf['distance_to_road'] <= 500, 1, 0)        

In [ ]:
gdf=gdf.drop(['tile_x', 'tile_y', 'geometry', 'distance_to_road'], axis=1)

In [ ]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio

land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(gdf['decimalLongitude'], gdf['decimalLatitude']))
gdf['easting_ni'], gdf['northing_ni'] = zip(*transformer_ni.itransform(coords))
gdf['easting_gb'], gdf['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class(row):
    try:
        # Try GB raster first
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
            # Read a small window around the point for better performance
            window = Window(col_idx, row_idx, 1, 1)
            land_cover_class = src.read(1, window=window)[0, 0]

            if land_cover_class == 0:  # Check NI raster if GB is 0
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
                    window = Window(col_idx, row_idx, 1, 1)
                    land_cover_class = src_ni.read(1, window=window)[0, 0]

        return land_cover_map.get(land_cover_class, "Unknown")
    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
gdf['Land_cover'] = gdf.progress_apply(get_land_cover_class, axis=1)


In [ ]:
gdf = gdf.dropna()
gdf=gdf.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
gdf = gdf[gdf['Land_cover'] != "Unknown"].reset_index(drop=True)
gdf.to_csv("tgb_bat_final_data.csv", index=False)